

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/CLASSIFICATION_EN_TREC.ipynb)




# **Classify text according to TREC classes**

## 0. Colab Setup

In [ ]:
!sudo apt-get install openjdk-8-jdk
!java -version
!pip install --ignore-installed -q pyspark==2.4.4
!pip install spark-nlp

In [2]:
import pandas as pd
import numpy as np
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
import json
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

## 1. Start Spark Session

In [3]:
spark = sparknlp.start()

## 2. Select the DL model and re-run all the cells below

In [4]:
### Select Model
#model_name = 'classifierdl_use_trec6'
model_name = 'classifierdl_use_trec50'

## 3. Some sample examples

In [5]:

text_list = [
    "What effect does pollution have on the Chesapeake Bay oysters?",
    "What financial relationships exist between Google and its advertisers?",
    "What financial relationships exist between the Chinese government and the Cuban government?",
    "What was the number of member nations of the U.N. in 2000?",
    "Who is the Secretary-General for political affairs?",
    "When did the construction of stone circles begin in the UK?",
    "In what country is the WTO headquartered?",
    "What animal was the first mammal successfully cloned from adult cells?",
    "What other prince showed his paintings in a two-prince exhibition with Prince Charles in London?",
    "Is there evidence to support the involvement of Garry Kasparov in politics?",
  ]

## 4. Define Spark NLP pipeline

In [ ]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

use = UniversalSentenceEncoder.pretrained(lang="en") \
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")


document_classifier = ClassifierDLModel.pretrained(model_name, 'en') \
          .setInputCols(["document", "sentence_embeddings"]) \
          .setOutputCol("class")

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 use,
 document_classifier
 ])


## 5. Run the pipeline

In [7]:
empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)
df = spark.createDataFrame(pd.DataFrame({"text":text_list}))
result = pipelineModel.transform(df)

## 6. Visualize results

In [ ]:
result.select(F.explode(F.arrays_zip('document.result', 'class.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("document"),
        F.expr("cols['1']").alias("class")).show(truncate=False)